In [ ]:
import scienceplots

import numpy as np
import matplotlib.pyplot as plt

from scipy.optimize import curve_fit
from scipy.integrate import solve_ivp

%matplotlib inline

plt.style.use('science')
plt.rcParams['figure.dpi'] = 220
plt.rcParams['figure.figsize'] = (6, 4)

# Some Python Arcana

Like most programing languages Python is not always intuitive; the language itself is over 30 years old and it carries with it the baggage of over 60 years of programming language design. Lets start by looking at a few examples of this baggage which can both trip you up and be used to your advantage.

## The Two Types of Types

Python types are generally either what is called "Mutable" or "Immutable", or what other languages call "Pass-by-Reference" or "Pass-by-Value" respectively. `int`, `float` , `str`, and `tuple` are all examples of "Immutable" types and variables of these types generally act the way you would expect; that is when you go to change the value of a variable it is replaced with the new value:

Variables containing "Mutable" are better thought of as bookmarks (or references) to the underlying data which they contain, the consequence of this being that multiple variables can contain the same bookmark, and changing the data behind one will be reflected when you use the other to look it up:

You can of course replace a bookmark with another and this is effectively the same operation when you change a variable which contains an Immutable type:

## Function Names == Variable Names

We often think of function names as a magical incantation, but really they're just a variable name that supports a `()` operation in the same way a list supports a `[]` operation. Let's see what this means in practice:

Which leads to two obvious questions:

We can also start to do crazy things like build functions to generate functions:

Now you may have realized by now that we can have an integer (say `3`) without ever giving it a variable name, we do this all the time when we do math in python. Can we do the same with functions?

Of course if we ever want to give them a name we can:

# Fitting Nonlinear Things

Now that we've done a bit of a python refresher lets try generating some data for a damped harmonic oscillator with $\omega_0 = 44000$, $\gamma = 4000$, and an offset of 5 and fitting a model to it:

In [ ]:
# We'll talk about whats going on here later if we have time

sol = solve_ivp(lambda _, y: [y[1], -(44000**2)*y[0] - 2*4000*y[1]], (0, 0.005), [1, 0], t_eval=np.linspace(0, 0.005, 1000))

t, x = sol.t, sol.y[0,::]
x_meas = x + np.random.normal(loc=0, scale=0.1, size=t.size) + 5

plt.plot(t, x_meas, label="Measured Data")
plt.plot(t, x + 5, label='Clean Data')

plt.legend()
plt.xlabel('Time')
plt.ylabel('Position')

In [ ]:
plt.plot(t, x_meas, label="Measured Data")


plt.legend()
plt.xlabel('Time')
plt.ylabel('Position')

Well that didn't work, lets use some of the things we developed to try and see why:

And lets plot the progression of the fit (getting darker as the fit progresses):

In [ ]:
plt.figure()
for i, p in enumerate(h):
    if i % 10:
        plt.plot(t, model(t, *p), c = (0.9*np.array([1, 1, 1])*(len(h)-i)/len(h))**2)
plt.ylim(0, 10)
plt.plot(t, x_meas, lw=0.2)

In [ ]:
plt.plot(t, model(t, *h[0]))
plt.plot(t, x_meas)

Looking at the progression of the fit can anyone tell me what happened? How do we correct it?

In [ ]:
plt.plot(t, x_meas)

In [ ]:
g0 = 1/0.0004
w0 = 2*np.pi/0.0002
o0 = 5.1
A0 = 0.9

p0 = [A0, g0, w0, o0]
plt.plot(t, x_meas)

p, pcov = curve_fit(model, t, x_meas, p0=p0, sigma=np.ones_like(t)*0.1, absolute_sigma=True)
plt.plot(t, model(t, *p))

print(list(p))
print(list(np.sqrt(np.diagonal(pcov))))

# What was that simulation thing?

Scipy has a handy tool called `solve_ivp`; It's a numerical integrator that can solve first order coupled differential equations. Lets look at the [documentation](https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.solve_ivp.html).

The differential equation for a damped harmonic oscillator is:

$$
\frac{d^2x}{dt^2} + 2\gamma \frac{dx}{dt} + \omega_0^2 x = 0
\implies \frac{d^2x}{dt^2} = - 2\gamma \frac{dx}{dt} - \omega_0^2 x
$$

But this is a 2nd-order ODE, can we rephrase it as a system of first-order ODEs?

Well if we rewrite things like this: 

$$
\frac{dx}{dt} = x^\prime
$$
$$
\frac{dx^\prime}{dt} = \frac{d^2x}{dt^2} = -2\gamma \frac{dx}{dt} - \omega_0^2 x = -2\gamma x^\prime - \omega_0^2 x
$$

Then we have two coupled first-order differential equations in the coordinates $x$ and $x^\prime$, if we write a function that takes these coordinates and returns their derivatives:

In [ ]:
ts = np.linspace(0, 25, 1000)
_, (ax1, ax2) = plt.subplots(2, sharex=True)



ax2.set_xlim(0, 25)

ax1.legend()
ax1.set_ylabel('Position')
ax2.set_ylabel('Velocity')
ax2.set_xlabel('Time')

plt.tight_layout()